### Approach for Cyclic Graphs

In [6]:
# # Bellman Ford approach using user defined data
# # This implementation might not avoid cyclic path traversals. It might visit a node multiple times...

# import math
# from collections import defaultdict

# class Edge:
#     def __init__(self, target, weight):
#         self.target = target
#         self.weight = weight

# class Graph:
#     def __init__(self, nodes):
#         self.nodes = nodes
#         self.edges = defaultdict(list)

# def rrp_fptas(G, s, t, C, epsilon):
#     n = len(G.nodes)
#     if n == 0:
#         return 0
#     delta = epsilon / (n - 1) if n > 1 else epsilon
    
#     labels = {node: {} for node in G.nodes}
#     labels[s][0] = (0, 0)  # (L, R)
    
#     for _ in range(n-1):
#         updated = False
#         new_labels = {node: {} for node in G.nodes}
#         for u in G.nodes:
#             if not labels[u]:
#                 continue
#             for interval_u in labels[u]:
#                 L_u, R_u = labels[u][interval_u]
#                 for edge in G.edges[u]:
#                     v = edge.target
#                     weight = edge.weight
#                     if weight > 0:
#                         r, l = weight, 0
#                     else:
#                         r, l = 0, -weight
#                     R_new = R_u + r
#                     L_new = L_u + l
#                     if L_new > C:
#                         continue
#                     if R_new == 0:
#                         interval = 0
#                     else:
#                         interval = int(math.log(R_new, 1 + delta))
#                     current_entry = new_labels[v].get(interval, (float('inf'), 0))
#                     if L_new < current_entry[0] or (L_new == current_entry[0] and R_new > current_entry[1]):
#                         new_labels[v][interval] = (L_new, R_new)
#                         updated = True
#         if not updated:
#             break
#         labels = new_labels
    
#     max_R = 0
#     max_L = 0
#     for interval in labels[t]:
#         L_val, R_val = labels[t][interval]
#         if L_val <= C and R_val > max_R:
#             max_R = R_val
#             max_L = L_val
#     return max_R, max_L

# # Example usage:
# nodes = ['n0', 'n1', 'n2']
# G = Graph(nodes)
# G.edges['n0'].append(Edge('n1', 7))
# G.edges['n1'].append(Edge('n2', -3))
# G.edges['n0'].append(Edge('n2', 2))



# print(rrp_fptas(G, 'n0', 'n2', C=3, epsilon=0.1))  

In [9]:
import math
from collections import defaultdict
import csv

class Label:
    def __init__(self, node, reward, penalty, visited, predecessor=None, last_edge=None):
        self.node = node
        self.reward = reward
        self.penalty = penalty
        self.visited = visited          # frozenset for hashability
        self.predecessor = predecessor  # Link to previous label
        self.last_edge = last_edge      # (u, v) edge

    def __repr__(self):
        return f"Label({self.node}, R={self.reward}, L={self.penalty})"

class Graph:
    def __init__(self):
        self.edges = defaultdict(list)  # {u: [(v, reward, penalty)]}

def load_graph_from_csv(filename):
    G = Graph()
    with open(filename, "r") as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        for u, v, weight in reader:
            weight = int(weight)
            reward = weight if weight > 0 else 0
            penalty = -weight if weight < 0 else 0
            G.edges[u].append((v, reward, penalty))
    return G

def rrp_fptas(G, s, t, C, epsilon):
    """SSMOSP-inspired FPTAS for RRP with cycle prevention."""
    nodes = list(G.edges.keys())
    n = len(nodes)
    delta = epsilon / (n - 1) if n > 1 else epsilon

    # Initialize labels: {node: {interval: best_label}}
    labels = defaultdict(dict)
    initial_visited = frozenset([s])
    labels[s][0] = Label(s, 0, 0, initial_visited)

    for _ in range(n-1):  # Bellman-Ford iterations
        new_labels = defaultdict(dict)
        for u in labels:
            for interval in labels[u]:
                label_u = labels[u][interval]

                # Skip if label is already dominated
                if u not in labels or interval not in labels[u]:
                    continue

                for (v, r, l) in G.edges[label_u.node]:
                    # Check for cycles
                    if v in label_u.visited:
                        continue

                    # Compute new reward and penalty
                    R_new = label_u.reward + r
                    L_new = label_u.penalty + l

                    if L_new > C:
                        continue

                    # Compute interval for R_new
                    if R_new == 0:
                        new_interval = 0
                    else:
                        new_interval = int(math.log(R_new, 1 + delta))

                    # Create new visited set
                    new_visited = label_u.visited.union(frozenset([v]))

                    # Create new label
                    new_label = Label(
                        node=v,
                        reward=R_new,
                        penalty=L_new,
                        visited=new_visited,
                        predecessor=label_u,
                        last_edge=(label_u.node, v)
                    )

                    # Update if new_label is better than existing in the interval
                    existing = new_labels[v].get(new_interval)
                    if not existing or (new_label.penalty < existing.penalty) or \
                       (new_label.penalty == existing.penalty and new_label.reward > existing.reward):
                        new_labels[v][new_interval] = new_label

        # Merge new labels into the main labels
        for v in new_labels:
            for interval in new_labels[v]:
                if interval not in labels[v] or \
                   new_labels[v][interval].penalty < labels[v][interval].penalty:
                    labels[v][interval] = new_labels[v][interval]

    # Extract the best path to t
    best_reward = 0
    best_label = None
    for interval in labels.get(t, {}):
        label = labels[t][interval]
        if label.penalty <= C and label.reward > best_reward:
            best_reward = label.reward
            best_label = label

    # Reconstruct path
    path = []
    current = best_label
    while current:
        path.append(current.node)
        current = current.predecessor
    path.reverse()

    return best_reward, best_label.penalty, path

# Usage Example
G = load_graph_from_csv("graph_data.csv")
max_reward, total_penalty, path = rrp_fptas(G, "n0", f"n{len(G.edges)-1}", C=20, epsilon=0.2)

if path:
    print(f"Path: {' -> '.join(path)}")
    print(f"Reward: {max_reward}, Penalty: {total_penalty}")
else:
    print("No valid path found.")

Path: n0 -> n4 -> n15 -> n13 -> n9 -> n14 -> n1 -> n7 -> n5 -> n6 -> n8 -> n16 -> n17 -> n18
Reward: 66, Penalty: 4
